In [1]:
import tensorflow as tf
from datetime import datetime

# keras API
keras API是tf.Module的多层次高级封装

## tf.keras.layers.Layer (inherit from tf.Module)
tf.keras.layers.Layer继承于tf.Module，做了一些的封装  
如果想使用这些功能，只需自定义keras layer，继承tf.keras.layers.Layer  

### 自定义keras layer
继承tf.keras.layers.Layer，并自定义call方法。  
(tf.keras.layers.Layer内部封装了__call__函数，它会调用用户自定义的call函数。所以可以用tf.function修饰call函数来得到静态图)  

In [2]:
class MyDense(tf.keras.layers.Layer):
    # adding **kwargs to support keras layer arguments
    # 在初始化一个实例的时候，就申请内存，创建变量w和b
    # 所以需要给定input dimension
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]
            ,name='w')
        )
        self.b = tf.Variable(
            tf.zeros([out_features])
            ,name='b'
        )
    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)
simple_layer = MyDense(name='simple', in_features=3, out_features=3)

2022-02-10 15:50:48.561090: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 15:50:48.580270: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe52ec0cb70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-10 15:50:48.580287: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [18]:
# 调用
simple_layer(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[2.8555908, 0.       , 0.       ]], dtype=float32)>

如果希望做到：  
拿到input data，再决定如何创建w和b变量，那么就需要做到  在第一次调用的时候，申请w和b的内存空间并创建tf.Variable  
如果使用tf.Module父类，使用内部变量is_built来甄别是否是第一次调用  
如果使用tf.keras.layers.Layer父类，可以在build方法中定义变量。该build方法只会call函数第一次调用的时候，被调用。  

In [3]:
class FlexibleDense(tf.keras.layers.Layer):
    # 实例化模型的时候，只需要传入output dim，不需要传入input dim
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
    # build方法，创建变量。会在第一次调用模型的时候调用build方法
    # build方法会自动传入一个变量名为 input_shape的变量，是input data的shape
    def build(self, input_shape):
        self.w = tf.Variable(
            tf.random.normal([input_shape[-1], self.out_features])
            ,name='w'
        )
        self.b = tf.Variable(
            tf.zeros([self.out_features]
            ,name='b')
        )
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
flexible_dense = FlexibleDense(out_features=3)

In [5]:
# 此时tf.Variables尚未创建
flexible_dense.variables

[]

In [6]:
flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]]))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-4.504209  ,  0.3593073 , -0.5687275 ],
       [-6.756314  ,  0.5389614 , -0.85309076]], dtype=float32)>

In [7]:
# 查看变量tf.Variables
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.15537804, -1.0250663 , -1.783119  ],
        [-0.57370484, -0.23540384, -0.12965351],
        [-1.5230217 ,  1.4401238 ,  1.6284087 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [9]:
# 对于该实例，build只会跑一次
try:
    print('module result: ', flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
    print("Failed: ", e)

Failed:  Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]


## tf.keras.Model (inherit from tf.keras.layers.Layer)
tf.keras.Model继承自tf.keras.layers.Layer，再做了额外的封装  
如果想使用这些额外的功能，只需自定义keras model，继承tf.keras.Model  

In [8]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [10]:
my_sequential_model = MySequentialModel(name="the_model")

In [14]:
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[ 5.22138   -1.8527935]], shape=(1, 2), dtype=float32)


In [23]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model_1/flexible_dense_3/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.6518539 ,  1.3783681 , -0.6574261 ],
        [-0.06599645, -0.01801451, -2.054874  ],
        [-0.47529003,  0.68516475,  1.8836492 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model_1/flexible_dense_3/Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model_1/flexible_dense_4/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.18086936,  0.6117754 ],
        [ 0.9098838 ,  0.2843005 ],
        [ 1.2947271 , -1.2913182 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model_1/flexible_dense_4/Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

目前来看，tf模型可以是多种类的混杂，包括tf.Module, tf.keras.layers.Layer, tf.keras.Model  
推荐用同一类tf模型：要么只用tf.Module，要么只用tf.keras  

In [15]:
my_sequential_model.summary()

Model: "my_sequential_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flexible_dense_1 (FlexibleDe multiple                  12        
_________________________________________________________________
flexible_dense_2 (FlexibleDe multiple                  8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


# 保存keras model
根据tf.Module类的用法，keras model可以使用tf.saved_model.save()以及tf.saved_model.load()来分别储存和读取  

keras model也定义了.save()方法，来保存model（graph文件和checkpoints）

In [12]:
my_sequential_model.save("../../model/exname_of_keras_file")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../../model/exname_of_keras_file/assets


2022-02-10 15:51:24.232585: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


keras model定义了tf.keras.models.load_model()方法，来读取model

In [13]:
reconstructed_model = tf.keras.models.load_model("../../model/exname_of_keras_file")

In [15]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 5.22138  , -1.8527935]], dtype=float32)>